# Solving the Graphene Defect Problem with Classical Brute Force

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.spatial.distance import pdist, squareform

import timeit
import itertools
import sys
import os
import copy
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

timestr = time.strftime("%Y%m%d-%H%M%S")

plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'font.size': 25})
plt.rc('xtick', labelsize=24) 
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.cif import CifWriter

from ase.visualize import view

from CRYSTALpytools.crystal_io import *
from CRYSTALpytools.convert import *
sys.path.insert(1,'../')
from quantum_computing_functions import *
from quantum_computing_postprocessing import *

import warnings
warnings.filterwarnings('ignore')

np.seterr(divide='ignore')
plt.style.use('tableau-colorblind10')

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import copy
import numpy as np

## Define graphene cell and instance of the problem

In [2]:
lattice = np.array([[ 1.233862, -2.137112,  0.      ],
                   [ 1.233862,  2.137112,  0.      ],
                   [ 0.      ,  0.      ,  8.685038]])

graphene = Structure(lattice, species=['C','C'], coords=[[2/3, 1/3, 0. ],[1/3, 2/3, 0.]])
graphene = SpacegroupAnalyzer(graphene).get_conventional_standard_structure()

num_vac = 3
lambda_1 = 3
n_supercell = 2

scaling_matrix = np.identity(3)*n_supercell
scaling_matrix[2][2] = 1
graphene_supercell = copy.deepcopy(graphene)
graphene_supercell.make_supercell(scaling_matrix)
structure = graphene_supercell
graphene_supercell.num_sites

Q = build_qubo_vacancies(graphene_supercell, num_vac=num_vac, coord_obj=False, lambda_1 = lambda_1, beta=0)
# Q

## Brute force algorithm

### This approach scales exponentially with the size of the QUBO matrix

In [1]:
repeats = 1


bitstrings = [np.binary_repr(i, len(Q)) for i in range(2 ** len(Q))]
costs = []
# this takes exponential time with the dimension of the QUBO
times = []
for i in range(repeats):
    start = timeit.default_timer()
    
    for b in bitstrings:
        z = np.array(list(b), dtype=int)
        cost = z.T @ Q @ z + ((Q.shape[0] - num_vac) ** 2) * lambda_1
        costs.append(cost)
    zipped = zip(bitstrings, costs)
    sort_zipped = sorted(zipped, key=lambda x: x[1])
    print(len(sort_zipped))
    print(sort_zipped[:200]) # can increase this number to see other solutions at no extra cost
    
    # Stop timer
    stop = timeit.default_timer()

    print('Time: ', stop - start)

    times.append(stop - start)

NameError: name 'Q' is not defined

In [6]:
np.mean(times)
np.std(times)
print(sort_zipped[-5:]) 

[('11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111101100', -183.0), ('11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110001', -183.0), ('11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110010', -183.0), ('11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110100', -183.0), ('11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111000', -183.0)]


## Now lets only check the solutions with exactly 3 vacancies 

In [7]:
##################
num_vac = 3
lambda_1 = 1
n_supercell = 8
##################

scaling_matrix = np.identity(3)*n_supercell
scaling_matrix[2][2] = 1
graphene_supercell = copy.deepcopy(graphene)
graphene_supercell.make_supercell(scaling_matrix)
structure = graphene_supercell
graphene_supercell.num_sites

Q = build_qubo_vacancies(graphene_supercell, num_vac=num_vac, coord_obj=False, lambda_1 = lambda_1, beta=0)
# Q

### This is the modified version of brute force used to verify large problems. This effectively changes the unconstained problem (with $2^n$ solutions) to a constrained problem (with $^{n}C_{N_{\text{vacancies}}}$ solutions).

In [8]:
repeats = 1

# Number of variables (assuming an 18-variable problem)


# Number of zeros we want in each bitstring
num_zeros = 3


time_avg_ls = []
time_std_ls = []
n_supercell_ls = []
n_vars_ls = []
min_E_ls = []

for k in range(1):
    times = []
    
    n_supercell = 8
    
    scaling_matrix = np.identity(3)*n_supercell
    scaling_matrix[2][2] = 1
    graphene_supercell = copy.deepcopy(graphene)
    graphene_supercell.make_supercell(scaling_matrix)
    structure = graphene_supercell
    graphene_supercell.num_sites
    
    Q = build_qubo_vacancies(graphene_supercell, num_vac=num_vac, coord_obj=False, lambda_1 = lambda_1, beta=0)
    
    n_vars = len(Q)
    
    # Generate all combinations of indices where zeros can be placed
    zero_combinations = itertools.combinations(range(n_vars), num_zeros)
    
    # Generate the relevant bitstrings with exactly `num_zeros` zeros
    bitstrings = []
    for zeros in zero_combinations:
        bitstring = ['1'] * n_vars
        for index in zeros:
            bitstring[index] = '0'
        bitstrings.append(''.join(bitstring))
    
    # Prepare to collect the costs
    costs = []
    
    # Run the brute force search with the filtered bitstrings
    
    for i in range(repeats):
        
        start = timeit.default_timer()
        
        for b in bitstrings:
            z = np.array(list(b), dtype=int)
            cost = z.T @ Q @ z + ((Q.shape[0] - num_vac) ** 2) * lambda_1
            costs.append(cost)
    
        # Zip and sort the results by cost
        zipped = zip(bitstrings, costs)
        sort_zipped = sorted(zipped, key=lambda x: x[1])
        
        # Print the top solutions
        print(n_supercell)
        print(len(sort_zipped))
        print(sort_zipped[:2])  # Increase this number to see other solutions if needed
        # print(sort_zipped[-2:]) 
        
        # Stop timer
        stop = timeit.default_timer()
    
        print('Time: ', stop - start)
    
        times.append(stop - start)

    time_avg = np.average(times)
    time_avg_ls.append(time_avg)
    time_std = np.std(times)
    time_std_ls.append(time_std)
    
    n_supercell_ls.append(n_supercell)
    n_vars_ls.append(n_vars)
    min_E = sort_zipped[0][1]
    min_E_ls.append(min_E)
    

df = pd.DataFrame({'n_supercell':[n_supercell_ls],
              'n_vars':[n_vars_ls],
              'min_E':[min_E_ls],
              'time_avg':[time_avg_ls],
                'time_std':[time_std_ls]
             })



path = f'RawResults/nsuper{n_supercell}_{timestr}.csv'
                
# Save the DataFrame to a CSV file
df.to_csv(path, index=False, float_format='%.15g')




8
341376
[('00111111111111111111111111111111111111111111111111111111111111111011111111111111111111111111111111111111111111111111111111111111', -185.0), ('01111110111111111111111111111111111111111111111111111111111111110111111111111111111111111111111111111111111111111111111111111111', -185.0)]
Time:  11.230014905333519


In [9]:
sort_zipped[0][1]

-185.0